In [163]:
import SAP_GUI
import win32com.client
import pandas as pd
import subprocess
from time import sleep
from datetime import datetime
from datetime import timedelta

"""----------------------------Inciar session----------------------------------------------------"""
# Insert User name and password
Keys=pd.read_excel("C:\\Users\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Escritorio\\SAP\\Claves\\Keys.xlsx")
user=Keys["User"][0]
password=Keys["Password"][0]
# Initialize session
session=SAP_GUI.SessionSAP(user,password)

In [36]:
"""----------------- Search ZD110 -------------------------------------------------------"""
Transsaccion_ZSD110="zsd110"
Varian_FIRME="PEN-FIRME"
Varian_MERCADEO="PEN-MERCADEO"
Varian_DISNAL_TOTAL="PENDIENTES"
provision_ZSD110="/REVISIONPEN"


In [144]:



"""----------------------------------------------CLEAN COLUMN AND DATA-----------------------------------------------------------"""
def Clean_Columns(List):
    Set_Columns=[i.strip() for i in List.columns]   #Alert in list
    clean_column=[i if "Unnamed" in i else None for i in List.columns]
    clean_column = list(filter(lambda x: x is not None, clean_column)) 
    List=List.set_axis(Set_Columns, axis=1).drop(clean_column,axis=1)  #Drop Unammed: 0
    return(List)

"""------------------------------------------FUNCION PARA CAMBIAR A NUMEROS-------------------------------------------------------"""

def Clean_num(x):
    x = float(str(x).strip().replace(',',''))
    return(x)

"""------------------------------------------FUNCION PARA COMPLETAR los 10 con 00 al inicio---------------------------------------"""
def Complete_00(lista):
    Nueva_Lista_Ordenes=[]
    for i in lista:
        while len(str(i))<10:
            i="0"+i
            # print(len(str(i)),i)
        Nueva_Lista_Ordenes.append(str(i))
    return(Nueva_Lista_Ordenes)



In [38]:
"""-----------------------------------BUSCAR y DESCARGAR TRANSSACCION ZD110---------------------------------------------------------------- """
# Pendiente Firme
Name_ZSD110_FIRME="Pendiente_Firme"  # Revisar suele no guardar
Search_ZSD110(Transsaccion_ZSD110,Varian_FIRME,provision_ZSD110)
SAP_GUI.Export_TXT2(Name_ZSD110_FIRME,session)

# Pendiente Mercadeo
Name_ZSD110_MERCADEO="Pendiente_Mercadeo"  # Revisar suele no guardar
Search_ZSD110(Transsaccion_ZSD110,Varian_MERCADEO,provision_ZSD110)
SAP_GUI.Export_TXT2(Name_ZSD110_MERCADEO,session)


# Pendiente Total 
Name_ZSD110_TOTAL="Pendiente_Total"  # Revisar suele no guardar
tabla_Total=Search_ZSD110(Transsaccion_ZSD110,Varian_DISNAL_TOTAL,provision_ZSD110)
SAP_GUI.Export_TXT2(Name_ZSD110_TOTAL,session)

In [376]:
tabla_zsd110=session.findById("wnd[0]/usr/cntlGRID1/shellcont/shell")     #Select table cont


In [11]:
def Download_ZSD110_Channels(tabla_zsd110,session):
    row_table=tabla_zsd110.RowCount
    List_Channels=[]
    for i in range(0,row_table-1):  #Se resta uno por los subtotales
        channelID=tabla_zsd110.GetCellValue(i,"VTWEG")
        tabla_zsd110.selectedRows = str(i)
        try:
            if channelID=="01":
                session.findById("wnd[0]/tbar[1]/btn[8]").press()
                session.findById("wnd[1]/tbar[0]/btn[45]").press()
                session.findById("wnd[1]/usr/subSUBSCREEN_STEPLOOP:SAPLSPO5:0150/sub:SAPLSPO5:0150/radSPOPLI-SELFLAG[1,0]").select()
                session.findById("wnd[1]/tbar[0]/btn[0]").press()
                session.findById("wnd[1]/usr/ctxtDY_PATH").text = "C:\\Users\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Escritorio\\SAP\\Archivos_CSV\\"   #CAMBIAR RUTA
                session.findById("wnd[1]/usr/ctxtDY_FILENAME").text = "Canal"+str(channelID)+"_ENT.txt"
                session.findById("wnd[1]/usr/ctxtDY_FILE_ENCODING").text = "4310"
                session.findById("wnd[1]/tbar[0]/btn[11]").press()
                session.findById("wnd[1]").close()
            session.findById("wnd[0]/tbar[1]/btn[7]").press()   
            session.findById("wnd[1]/tbar[0]/btn[45]").press()
            session.findById("wnd[1]/usr/subSUBSCREEN_STEPLOOP:SAPLSPO5:0150/sub:SAPLSPO5:0150/radSPOPLI-SELFLAG[1,0]").select()
            session.findById("wnd[1]/tbar[0]/btn[0]").press()
            session.findById("wnd[1]/usr/ctxtDY_PATH").text = "C:\\Users\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Escritorio\\SAP\\Archivos_CSV\\"   #CAMBIAR RUTA
            session.findById("wnd[1]/usr/ctxtDY_FILENAME").text = "Canal"+str(channelID)+"_PEN.txt"
            session.findById("wnd[1]/usr/ctxtDY_FILE_ENCODING").text = "4310"
            session.findById("wnd[1]/tbar[0]/btn[11]").press()
            session.findById("wnd[1]").close()
            List_Channels.append(channelID)
            print("Se encontro detalle Channel ",channelID)
        except:
            print("No se encontro ",channelID)
    return(List_Channels)


In [12]:
List_Channels=Download_ZSD110_Channels(tabla_Total,session)

Se encontro detalle Channel  01
Se encontro detalle Channel  02
Se encontro detalle Channel  07
Se encontro detalle Channel  08
No se encontro  12
Se encontro detalle Channel  14
Se encontro detalle Channel  15
Se encontro detalle Channel  19
No se encontro  50


In [14]:
# ZSD110 FIRME
url_ZSD110_FIRME="C:\\Users\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Documentos 1\\SAP\\SAP GUI\\"+str(Name_ZSD110_FIRME)+".txt"
Data_ZSD110_FIRME=pd.read_csv(url_ZSD110_FIRME,skiprows=1,delimiter="\t")
Data_ZSD110_FIRME=Clean_Columns(Data_ZSD110_FIRME)

In [15]:
# ZSD110 MERCADE0
url_ZSD110_MERCADEO="C:\\Users\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Documentos 1\\SAP\\SAP GUI\\"+str(Name_ZSD110_MERCADEO)+".txt"
Data_ZSD110_MERCADEO=pd.read_csv(url_ZSD110_MERCADEO,skiprows=1,delimiter="\t")
Data_ZSD110_MERCADEO=Clean_Columns(Data_ZSD110_MERCADEO)

In [16]:
# ZSD110 Pendiente Total
url_ZSD110_PENDIENTES="C:\\Users\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Documentos 1\\SAP\\SAP GUI\\"+str(Name_ZSD110_TOTAL)+".txt"
Data_ZSD110_PENDIENTES=pd.read_csv(url_ZSD110_PENDIENTES,skiprows=1,delimiter="\t")
Data_ZSD110_PENDIENTES=Clean_Columns(Data_ZSD110_PENDIENTES)

In [17]:
"""---------------------CAMBIAR INDICES------------------------------------------"""
Channels_Name={1:"MODERNO",2:"TRADICIONAL",7:"TIENDAS DE BELLEZA",8:"PROFESIONAL",12:"INSTITUCIONAL",14:"HARD DISCOUNTERS",15:"ECOMMERCE",19:"ALTERNATIVO",50:"ALMACENES INTERNOS"}


Data_ZSD110_MERCADEO["Can.distr."]=Data_ZSD110_MERCADEO["Can.distr."].replace(Channels_Name)
Data_ZSD110_FIRME["Can.distr."]=Data_ZSD110_FIRME["Can.distr."].replace(Channels_Name)
Data_ZSD110_PENDIENTES["Can.distr."]=Data_ZSD110_PENDIENTES["Can.distr."].replace(Channels_Name)


In [234]:
import pandas as pd
import openpyxl
import numpy
from openpyxl.styles import Font, Alignment, PatternFill
from openpyxl.styles import Border, Side

wb = openpyxl.Workbook()
hoja = wb.active
# Añade la hoja 'Hoja' al final (por defecto)
hoja.title = "Pendientes"

"""------------------------CREATE HOJA------------------------"""
#wb.create_sheet("Hoja")
"""-----------------------------------------------------------"""

print(wb.sheetnames)


nombresCeldas = Data_ZSD110_FIRME.columns
color='00a135'
"""------------------------------------------Funcion para generar estilos--------------------------------------------------------------"""
def Style_sheet(nombresCeldas,color=None):
    for fila in hoja.iter_rows():
        for celda in fila:
            if celda.value in nombresCeldas:
                celdaEstilos = hoja[celda.coordinate]
                celdaEstilos.font = Font(name='Arial', size=10, bold=True)
                celdaEstilos.alignment = Alignment(horizontal='center', vertical='center')
                celdaEstilos.border= Border(left=Side(style='thin'), right=Side(style='thin'), top=Side(style='thin'), bottom=Side(style='thin'))
                if color==None:
                    None
                else:
                    celdaEstilos.fill = PatternFill(start_color=color, end_color=color, fill_type='solid')
                print("El nombre se encontró en la celda:", celda.coordinate," Name:",celda.value)


"""------------------------------------------Copiar y crear archivo Excel--------------------------------------------------------------"""
hoja.append(("","","","",""))
hoja.append((tuple(Data_ZSD110_FIRME.columns)))
for i,j in Data_ZSD110_FIRME.iterrows():
    hoja.append((j["Can.distr."],j["Pedidos"],j["Lineas"],j["Unidades"],j["Valor Neto"]))
    Style_sheet([j["Can.distr."],j["Pedidos"],j["Lineas"],j["Unidades"],j["Valor Neto"]])

hoja.append(("","","","",""))
hoja.append((tuple(Data_ZSD110_MERCADEO.columns)))
for i,j in Data_ZSD110_MERCADEO.iterrows():
    hoja.append((j["Can.distr."],j["Pedidos"],j["Lineas"],j["Unidades"],j["Valor Neto"]))
    Style_sheet([j["Can.distr."],j["Pedidos"],j["Lineas"],j["Unidades"],j["Valor Neto"]])

hoja.append(("","","","",""))
hoja.append((tuple(Data_ZSD110_PENDIENTES.columns)))
for i,j in Data_ZSD110_PENDIENTES.iterrows():
    hoja.append((j["Can.distr."],j["Pedidos"],j["Lineas"],j["Unidades"],j["Valor Neto"])) 
    Style_sheet([j["Can.distr."],j["Pedidos"],j["Lineas"],j["Unidades"],j["Valor Neto"]])


Style_sheet(nombresCeldas,color)


['Pendientes']
El nombre se encontró en la celda: A3  Name: MODERNO
El nombre se encontró en la celda: B3  Name: 307
El nombre se encontró en la celda: C3  Name:  1,418
El nombre se encontró en la celda: D3  Name:   60,393
El nombre se encontró en la celda: E3  Name:   913,884,300
El nombre se encontró en la celda: A4  Name: TRADICIONAL
El nombre se encontró en la celda: B4  Name: 36
El nombre se encontró en la celda: C4  Name:    426
El nombre se encontró en la celda: D4  Name:   13,351
El nombre se encontró en la celda: E4  Name:   131,560,900
El nombre se encontró en la celda: A5  Name: TIENDAS DE BELLEZA
El nombre se encontró en la celda: B5  Name: 47
El nombre se encontró en la celda: C5  Name:    797
El nombre se encontró en la celda: D5  Name:    4,263
El nombre se encontró en la celda: E5  Name:    40,236,800
El nombre se encontró en la celda: A6  Name: PROFESIONAL
El nombre se encontró en la celda: B6  Name: 3
El nombre se encontró en la celda: C6  Name:      4
El nombre se en

In [20]:
Tabla_reporte=pd.merge(pd.merge(Data_ZSD110_PENDIENTES,Data_ZSD110_FIRME,how="outer",on="Can.distr."),Data_ZSD110_MERCADEO,how="outer",on="Can.distr.")

,Can.distr.,Pedidos_x,Lineas_x,Unidades_x,Valor Neto_x,Pedidos_y,Lineas_y,Unidades_y,Valor Neto_y,Pedidos,Lineas,Unidades,Valor Neto
0,MODERNO,868,"8,625","89,896","1,240,546,500",403.0,"1,962","65,831","1,014,185,500",311.0,679,"9,005",0.0
1,TRADICIONAL,125,801,"18,518","97,990,000",44.0,322,"8,555","95,154,100",79.0,466,"9,731",0.0
2,TIENDAS DE BELLEZA,108,"1,999","15,908","80,601,100",77.0,"1,489","8,186","80,601,100",31.0,510,"7,722",0.0
3,PROFESIONAL,42,251,"1,565","43,412,800",35.0,237,"1,273","79,110,200",8.0,17,508,0.0
4,INSTITUCIONAL,6,13,"3,890","31,465,400",6.0,13,"3,890","31,465,400",NaN,NaN,NaN,NaN
5,HARD DISCOUNTERS,4,44,"20,142","134,286,400",4.0,44,"20,142","134,286,400",NaN,NaN,NaN,NaN
6,ECOMMERCE,22,70,81,"1,992,100",32.0,98,113,"2,834,500",NaN,NaN,NaN,NaN
7,ALTERNATIVO,2,111,"7,966","48,800,100",2.0,114,"8,070","49,570,600",0.0,0,0,0.0
8,ALMACENES INTERNOS,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,"1,177","11,914","157,966","1,679,094,400",603.0,"4,279","116,060","1,487,207,800",429.0,"1,672","26,966",0.0


In [235]:
wb.save('C:\\Users\\prac.ingindustrial2\\Downloads\\Permanencia.xlsx') 

In [255]:
"""---------------------------------------BUSCAR CLIENTE EXITO EN VA05N-------------------------------"""

Transsaccion="va05n"
provision="PEDIDOS"
variant="ADESPACHOS"        
client_Exito="1000000000"   #Cliente Exito
date="27.05.2023"           #Fecha

def Search_VA05N(Transsaccion,provision,variant,client,date):
    session.StartTransaction(Transsaccion)
    session.findById("wnd[0]/tbar[1]/btn[17]").press()
    session.findById("wnd[1]/usr/txtV-LOW").text = provision
    session.findById("wnd[1]/usr/txtENAME-LOW").text = variant
    session.findById("wnd[1]/usr/txtV-LOW").caretPosition = 8
    session.findById("wnd[1]/tbar[0]/btn[8]").press()
    session.findById("wnd[0]/usr/ctxtSKUNNR-LOW").text = client
    session.findById("wnd[0]/usr/ctxtSAUDAT-LOW").text= date
    session.findById("wnd[0]/tbar[1]/btn[8]").press()

# Buscar pepidos de cliente Exito

Search_VA05N(Transsaccion,provision,variant,client_Exito,date)

In [256]:
# Descargar archivo del cliente Exito

Name_VA05N="Cliente_Exito"
Ruta_VA05N="C:\\Users\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Escritorio\\SAP\\Archivos_CSV\\"

SAP_GUI.Export_TXT2(Name_VA05N,session,Ruta_VA05N)

In [258]:
#Leer achivo Cliente Exito
import pandas as pd
Ruta_VA05N="C:\\Users\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Escritorio\\SAP\\Archivos_CSV\\"+str(Name_VA05N)+".txt"
data_Exito = pd.read_csv(Ruta_VA05N, skiprows=1, delimiter='\t')
data_Exito=Clean_Columns(data_Exito)

In [313]:
Numero_Pedido=data_Exito.Pedido[0:12].to_clipboard(index=False, header=False)



In [315]:
"""------------------------BUSCAR PEPIDOS EN LA ZSD110-----------------"""
Transsaccion="zsd10"
provision=None
session.StartTransaction(Transsaccion)
session.findById("wnd[0]/usr/btn%_SP$00013_%_APP_%-VALU_PUSH").press()
session.findById("wnd[1]/tbar[0]/btn[24]").press()
session.findById("wnd[1]/tbar[0]/btn[8]").press()
if provision!=None:
    session.findById("wnd[0]/usr/ssub%_SUBSCREEN_TOPBLOCK:PPIO_ENTRY:1100/ctxtPPIO_ENTRY_SC1100-ALV_VARIANT").text = provision
session.findById("wnd[0]/tbar[1]/btn[8]").press()


In [323]:
"--------------EXPORTAR DATOS------------------------------"
Name_Pedidos="Pedidos_Exito"
ruta_Pedidos="C:\\Users\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Escritorio\\SAP\\Archivos_CSV\\"
SAP_GUI.Export_TXT2(Name_Pedidos,session,ruta_Pedidos)



In [335]:
Pedidos_Exito=pd.read_csv("C:\\Users\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Escritorio\\SAP\\Archivos_CSV\\"+str(Name_Pedidos)+".txt",skiprows=5,delimiter='\t')

In [357]:
Pedidos_Exito=Clean_Columns(Pedidos_Exito)
# Eliminar espacios en blanco
Pedidos_Exito=Pedidos_Exito[Pedidos_Exito['Doc.venta'].notnull()]
Documento_Venta=Pedidos_Exito['Doc.venta'].astype(int)

10 000Pb00309
10 00pndsfsdf


['000Pb00309', '00pndsfsdf', '0pb0023456', 'Pb003093665']

# Aqui se copia el canal 01

In [164]:
Canal01_Pendiente=pd.read_csv("C:\\Users\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Documentos 1\\SAP\\SAP GUI\\Canal01_PEN.txt",skiprows=1,delimiter="\t")
Canal01_Pendiente=Clean_Columns(Canal01_Pendiente)

# Filtro para excluir a los clientes Exito y 
Canal01_Entregado=pd.read_csv("C:\\Users\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Documentos 1\\SAP\\SAP GUI\\Canal01_ENT.txt",skiprows=1,delimiter="\t")
Canal01_Entregado=Clean_Columns(Canal01_Entregado)

In [165]:
"""""------Limpio los datos-----------------------------------"""
Canal01_Entregado['Valor Neto']=Canal01_Entregado['Valor Neto'].apply(lambda x: Clean_num(x))
Canal01_Entregado['Cantidad E']=Canal01_Entregado['Cantidad E'].apply(lambda x: Clean_num(x))

In [166]:
#Lineas Totales
Lines_Canal01_Pendiente=len(Canal01_Pendiente)
Lines_Canal01_Entregado=len(Canal01_Entregado)

#Agenda Exito y Cencosub
Agenda_Exito=["0085","0020","0146","0149","0050","0138","0045"]
Agenda_Cencosub=["0093","0122","0127","0095","0060"]
(Agenda_Exito+Agenda_Cencosub)

['0085',
 '0020',
 '0146',
 '0149',
 '0050',
 '0138',
 '0045',
 '0093',
 '0122',
 '0127',
 '0095',
 '0060']

In [167]:
# Ecluir Agenda de pedidos exito
Exluidos_Entrega=set()
for i in Canal01_Entregado['Pedido.1']:
    if (str(i)[:4] in Agenda_Exito) or (str(i)[:3] in Agenda_Exito):
        Exluidos_Entrega.add(i)


Filtro_Canal01_Entregado=(Canal01_Entregado['Pedido.1'].isin(Exluidos_Entrega))   #Excluyo con ~
Canal01_Entregado_Exluido=Canal01_Entregado.loc[Filtro_Canal01_Entregado].reset_index(drop=True)
Lines_Canal01_Entregado_Exluido=len(Canal01_Entregado_Exluido)



In [168]:
Canal01_Entregado_Exluido.columns


Index(['Can.distr.', 'Denomin.', 'Seccion', 'Estado', 'Solic.', 'Nombre',
       'Entrega', 'Posicion E', 'F.Creacion', 'Pedido', 'Clase Pedi',
       'F.Creacion.1', 'Pedido.1', 'Posicion P', 'Material', 'Moneda',
       'Cantidad E', 'UM venta', 'Valor Neto', 'StTotPick.', 'StatGlWM',
       'StTotMovMe', 'Stat.fact.', 'Status PT'],
      dtype='object')

In [169]:
#Canal01_Entregado_Exluido.pivot_table(,,['Material', 'Valor Neto'])

table_dinamica_Exito_entrega=Canal01_Entregado_Exluido.pivot_table(index=['Pedido.1','Clase Pedi'],aggfunc={'Material':'count','Cantidad E':sum,'Valor Neto':sum}).reset_index()
Lineas_table_dinamica_Exito_entrega=sum(table_dinamica_Exito_entrega["Material"])
Lineas_table_dinamica_Exito_entrega

3124

In [170]:
table_dinamica_Exito_entrega['Pedido.1'].to_clipboard(header=False,index=False)



In [171]:
"""----------------------------------------------------------------Buscar en ZSD037---------------------------------------"""

Transsaccion_ZSD037="zsd037"


def Search_Pedidos_ZSD037(Transsaccion,Series,session,provision=None):      #(column Dataframe)
    session.StartTransaction(Transsaccion)
    Series=Series.to_clipboard(index=False, header=False)
    if provision!=None:
        session.findById("wnd[0]/usr/ctxt%LAYOUT").text = provision
    session.findById("wnd[0]/usr/btn%_SP$00011_%_APP_%-VALU_PUSH").press()
    session.findById("wnd[1]/tbar[0]/btn[24]").press()
    session.findById("wnd[1]/tbar[0]/btn[8]").press()
    if provision!=None:
        session.findById("wnd[0]/usr/ctxt%LAYOUT").text = provision
    session.findById("wnd[0]/tbar[1]/btn[8]").press()


In [172]:
Search_Pedidos_ZSD037(Transsaccion_ZSD037,table_dinamica_Exito_entrega['Pedido.1'],session)

In [173]:
"""------------------------------SAVE ZSD037--------------------------------------------"""

Name_ZSD037="Pedidos pendientes"
Ruta_ZSD037="C:\\Users\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Escritorio\\SAP\\Archivos_CSV\\"
SAP_GUI.Export_TXT2(Name_ZSD037,session,Ruta_ZSD037)

In [154]:
# ZSD110 FIRME
url_ZSD037_Pedidos="C:\\Users\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Escritorio\\SAP\\Archivos_CSV\\"+str(Name_ZSD037)+".txt"
Data_ZSD110_Pedidos_Pendientes=pd.read_csv(url_ZSD037_Pedidos,skiprows=5,delimiter="\t")

Data_ZSD110_Pedidos_Pendientes=Clean_Columns(Data_ZSD110_Pedidos_Pendientes)
Data_ZSD110_Pedidos_Pendientes=Data_ZSD110_Pedidos_Pendientes[Data_ZSD110_Pedidos_Pendientes['Nº pedido cliente'].notnull()]   #Elimina filas nulas
Data_ZSD110_Pedidos_Pendientes['Nº pedido cliente']=Data_ZSD110_Pedidos_Pendientes['Nº pedido cliente'].astype(int).astype(str)


Column_Stand_Pedidos_Pendientes=Data_ZSD110_Pedidos_Pendientes.columns

Column_Stand_Pedidos_Pendientes

Data_ZSD110_Pedidos_Pendientes_Fecha=Data_ZSD110_Pedidos_Pendientes[['Nº pedido cliente','PrimFeEntr','ÚltEntrega']].drop_duplicates()


In [155]:
Remplace_Pedidos_Pendientes_Fecha=Complete_00(Data_ZSD110_Pedidos_Pendientes_Fecha['Nº pedido cliente'])  #Se pasa una columna pero en formato Str 

In [182]:
Data_ZSD110_Pedidos_Pendientes_Fecha['Nº pedido cliente']=Remplace_Pedidos_Pendientes_Fecha
Data_ZSD110_Pedidos_Pendientes_Fecha

,Nº pedido cliente,PrimFeEntr,ÚltEntrega
0,0020631946,01.06.2023,01.06.2023
87,0050432800,03.06.2023,03.06.2023
97,0050432811,03.06.2023,03.06.2023
387,0050432812,03.06.2023,03.06.2023
405,0050432813,03.06.2023,03.06.2023
415,0050432797,03.06.2023,03.06.2023
425,0050432798,03.06.2023,03.06.2023
586,0050432814,03.06.2023,03.06.2023
596,0050432815,03.06.2023,03.06.2023
645,0050432822,03.06.2023,03.06.2023


In [183]:
Tabla_total=pd.merge(Data_ZSD110_Pedidos_Pendientes_Fecha,table_dinamica_Exito_entrega,how="right",left_on="Nº pedido cliente",right_on="Pedido.1")
Tabla_total

,Nº pedido cliente,PrimFeEntr,ÚltEntrega,Pedido.1,Clase Pedi,Cantidad E,Material,Valor Neto
0,0020631946,01.06.2023,01.06.2023,0020631946,ZVMI,14.0,11,337034.0
1,0050432797,03.06.2023,03.06.2023,0050432797,ZED1,19.0,2,819895.0
2,0050432798,03.06.2023,03.06.2023,0050432798,ZED1,20.0,2,1291940.0
3,0050432800,03.06.2023,03.06.2023,0050432800,ZED1,30.0,2,404418.0
4,0050432811,03.06.2023,03.06.2023,0050432811,ZVMI,2270.0,724,39626805.0
5,0050432812,03.06.2023,03.06.2023,0050432812,ZVMI,2.0,2,26218.0
6,0050432813,03.06.2023,03.06.2023,0050432813,ZVMI,893.0,437,7897508.0
7,0050432814,03.06.2023,03.06.2023,0050432814,ZVMI,14.0,8,438721.0
8,0050432815,03.06.2023,03.06.2023,0050432815,ZVMI,486.0,293,3757894.0
9,0050432822,03.06.2023,03.06.2023,0050432822,ZEDI,188.0,18,3198056.0


In [184]:
""""-----------------------EXCLUIR EXITO Y SENCOSU-------------------------------------------------------------------------"""
Filtro_Canal01_Pendiente_Excluidos=(Canal01_Pendiente["Solic."]!=1000000000) & (Canal01_Pendiente["Solic."]!=1000007000)
Canal01_Pendiente_Excluidos=Canal01_Pendiente.loc[~Filtro_Canal01_Pendiente_Excluidos].reset_index(drop=True)

In [185]:
Canal01_Pendiente_Excluidos

,Can.distr.,Denomin.,Seccion,Estado,Solic.,Nombre,Pedido,Clase Pedi,Creado el,Pedido.1,...,Moneda,Cantidad P,Cantidad C,UM venta,Vlr.Neto C,Status Glo,Status Tot,Status Ent,Status Ent.1,Status Ent.2
0,1,MODERNO,Picking,1. Pendiente de generación de Entrega,1000000000,ALMACENES EXITO SA,903118407,ZEDI,26.05.2023,0050432856,...,COP,12,0,UN,0,B,A,B,B,A
1,1,MODERNO,Picking,2. Entrega generada parcialmente,1000000000,ALMACENES EXITO SA,903118413,ZED1,26.05.2023,0050432798,...,COP,12,4,UN,"371,648",B,A,B,B,B
2,1,MODERNO,Picking,1. Pendiente de generación de Entrega,1000000000,ALMACENES EXITO SA,903118434,ZEDI,26.05.2023,0050432858,...,COP,12,0,UN,0,B,A,B,B,A
3,1,MODERNO,Picking,1. Pendiente de generación de Entrega,1000000000,ALMACENES EXITO SA,903118437,ZEDI,26.05.2023,0050432856,...,COP,12,0,UN,0,B,A,B,B,A
4,1,MODERNO,Picking,1. Pendiente de generación de Entrega,1000000000,ALMACENES EXITO SA,903118438,ZEDI,26.05.2023,0050432856,...,COP,12,5,UN,"70,630",B,A,B,B,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
917,1,MODERNO,Picking,1. Pendiente de generación de Entrega,1000000000,ALMACENES EXITO SA,903122988,ZVMI,31.05.2023,0146313654,...,COP,5,5,UN,"134,120",B,A,A,A,A
918,1,MODERNO,Picking,1. Pendiente de generación de Entrega,1000000000,ALMACENES EXITO SA,903122989,ZVMI,31.05.2023,0085751310,...,COP,1,0,UN,0,B,A,B,B,A
919,1,MODERNO,Picking,1. Pendiente de generación de Entrega,1000000000,ALMACENES EXITO SA,903122989,ZVMI,31.05.2023,0085751310,...,COP,3,0,UN,0,B,A,B,B,A
920,1,MODERNO,Picking,1. Pendiente de generación de Entrega,1000000000,ALMACENES EXITO SA,903122989,ZVMI,31.05.2023,0085751310,...,COP,2,0,UN,0,B,A,B,B,A


In [186]:
Canal01_Pendiente['Cantidad P'].sum()

'         2         1         2         3         1         1         1         2         2         2         1         2         1         2         2         1         3         2         2         1         1         2         1         1         3         1         1         2         2         2         2         1         1         3         3         1         1         1         2         3         1         2         2         1         3         2         3         2         2         2         1         2         3         1         1         3         3         1         3         3         2         3         1         1         3         1         1         3         9         1         1         1         3         2         2         2         2         1         2         3         1         1         1         1         3         1         2         2         1         3         2         1         1         3         1         1         1         2         1         

In [187]:
Canal01_Pendiente['Cantidad C'].sum()

9472

In [191]:
from dash import Dash, dash_table
import pandas as pd

df = Tabla_total

app = Dash(__name__)

app.layout = dash_table.DataTable(df.to_dict('records'), [{"name": i, "id": i} for i in df.columns])

if __name__ == '__main__':
    app.run_server(debug=True)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: on


SystemExit: 1

c:\Users\prac.ingindustrial2\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3513: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [42]:
import pandas as pd
import numpy as np

colums=["Exito","Cencosub"]
data=np.array([[1,2,3],[3,4,5]])

pd.DataFrame(data,index=["Exito","Cencosub"],columns=["Cliente","ZVMI","FIRME"])



,Cliente,ZVMI,FIRME
Exito,1,2,3
Cencosub,3,4,5


In [201]:
data = [
    {'Nombre': 'John', 'Edad': 25, 'Ciudad': 'Nueva York'},
    {'Nombre': 'Jane', 'Edad': 30, 'Ciudad': 'Los Ángeles'},
    {'Nombre': 'Mark', 'Edad': 35, 'Ciudad': 'Chicago'},
    {'Nombre': 'Emily', 'Edad': 28, 'Ciudad': 'San Francisco'}
]

# Opciones para el filtro de ciudad
ciudades = [{'value': ciudad} for ciudad in set(d['Ciudad'] for d in data)]

ciudades

[{'value': 'Nueva York'},
 {'value': 'San Francisco'},
 {'value': 'Los Ángeles'},
 {'value': 'Chicago'}]

# Nota falta cambiar etiquetas canales 01,02,03